In [1]:
import numpy as np
import cvxpy as cp

from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

In [2]:
from models.LISTA_CPSS import LISTA_CPSS
from utils.train import train
from utils.synthetic_data import SyntheticSignals

In [3]:
torch.cuda.is_available()

True

In [4]:
M = 10
LAYERS = 16
BETA_INIT = 0.4
EPOCHS = 50
SNR = None

LISTA_CPSS_models = {
    m: None for m in range(M)
}

for m in tqdm(range(M)):
    # Dimensions for the signal, its sparsity and its observation

    n_ = 500
    m_ = 250
    p_ = 0.1

    # Sensing matrix

    A_ = torch.normal(0, torch.sqrt(torch.tensor(1/m_)), size = (m_, n_))
    A_ /= torch.linalg.norm(A_, dim = 0)

    # Building a training set and a test set 
    train_set = SyntheticSignals(
        A = A_,
        n = n_,        
        m = m_,
        p = p_,
        SNR = SNR,
        size = 4000
        ).set_loader()

    test_set = SyntheticSignals(
        A = A_,
        n = n_,        
        m = m_,
        p = p_,
        SNR = SNR,
        size = 1000
        ).set_loader()
    
    model = LISTA_CPSS(torch.clone(A_), BETA_INIT, LAYERS)
    train(model, train_set, test_set, num_epochs=50, verbose = False)

    LISTA_CPSS_models[m] = model

100%|██████████| 10/10 [04:26<00:00, 26.69s/it]


In [8]:
model_ = LISTA_CPSS_models[0]

In [ ]:
[torch.linalg.norm(model_.W)]